In [58]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
import sys
import os
sys.path.append(os.path.join(os.path.split(os.path.abspath(''))[0], 'project/packages'))

In [60]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets

import data_types
import stock_data_manager
import stock_data_consumer

sdm = stock_data_manager.StockDataManager()
sdm._testing = True
sdm.run()

sdc = stock_data_consumer.StockDataConsumer(all_symbols=sdm.all_symbols,
                                            stock_categories=sdm.stock_categories,
                                            category_allocations=sdm.category_allocations,
                                            index_tracker_stocks=sdm.index_tracker_stocks,
                                            watchlist_stocks=sdm.watchlist_stocks,
                                            portfolio_stocks=sdm.portfolio_stocks,
                                            positions=sdm.positions)
sdc.run()

portfolio_market_dates = sdc.portfolio_market_dates
# Pick last date for any outputs
date = portfolio_market_dates[len(portfolio_market_dates)-1]

portfolio_stock_stats_combined = sdc.get_portfolio_stock_stats(combined=True)
portfolio_aggregated_stats = sdc.get_portfolio_aggregate_stats()
portfolio_stock_composition_stats = sdc.get_portfolio_stock_composition_stats()
portfolio_category_composition_stats = sdc.get_portfolio_category_composition_stats()
portfolio_category_composition_stats_combined = sdc.get_portfolio_category_composition_stats(combined=True)

In [61]:
fig = px.line(portfolio_stock_stats_combined, x='date', y="total_pct_gain", color="symbol",
              hover_name="symbol", hover_data=['invested_amount', 'market_value', 'unrealized_gain', 
                                               'unrealized_pct_gain', 'realized_gain', 'realized_pct_gain', 
                                               'total_gain', 'total_pct_gain', 'open', 'close', 'high', 
                                               'low', 'volume', 'quantity', 'average_cost', 'company_name'],
              title='Portfolio Stocks')
fig.show()

In [62]:
fig = px.line(portfolio_aggregated_stats, x='date', y="total_pct_gain", 
              hover_data=['invested_amount', 'market_value', 'unrealized_gain', 
                          'unrealized_pct_gain', 'realized_gain', 'realized_pct_gain', 
                          'total_gain', 'total_pct_gain'],
#               color='invested_amount',
              title='Portfolio Value')
fig.show()

In [64]:
s_composition_stats = portfolio_stock_composition_stats[date]
fig = px.bar(s_composition_stats.sort_values(by='total_pct_gain', ascending=False),
             x="total_pct_gain", y="symbol", 
             title='Total Gain by Stock', color="symbol",
             text="total_pct_gain",
             hover_data=['company_name'],
             orientation='h',
             height=700)
fig.show()

In [ ]:
fig = px.pie(s_composition_stats, values='composition_invested', names='symbol', hover_name='symbol',
             hover_data=['invested_amount'], title='Stock Allocation')
fig.show()

In [ ]:
fig = px.pie(s_composition_stats, values='relative_total_gain', names='symbol', hover_name='symbol',
             hover_data=['total_gain'], title='Relative Gain by Stock')
fig.show()

In [ ]:
c_composition_stats = portfolio_category_composition_stats[date]
fig = px.pie(c_composition_stats, values='composition_invested', names='category',
             hover_data=['invested_amount'], title='Allocation by Category')
fig.show()

In [ ]:
fig = px.pie(c_composition_stats, values='relative_total_gain', names='category',
             hover_data=['total_gain'], title='Relative Gain by Category')
fig.show()

In [ ]:
fig = px.bar(c_composition_stats, x="desired_allocation", y="category", 
             title='Desired Allocation', color="category", text="desired_allocation", 
             orientation='h')
fig.show()

In [ ]:
fig = px.bar(c_composition_stats, x="desired_composition_invested_diff", y="category", 
             title='Allocation Difference', color="category", text="desired_composition_invested_diff", 
             orientation='h')
fig.show()

In [ ]:
allocation_solution = sdc.maximize_desired_allocation(date=date)
fig = px.bar(allocation_solution, x="allocation_break_even", y="category", 
             title='Allocation Break Even', color="category", text="allocation_break_even", 
             orientation='h')
fig.show()

In [ ]:
fig = px.bar(c_composition_stats, x="total_pct_gain", y="category", 
             title='Total Gain by Category', color="category", text="total_pct_gain",
             orientation='h')
fig.show()

In [ ]:
fig = px.bar(portfolio_category_composition_stats_combined, x="total_pct_gain", y="category", 
             title='Total Gain by Category Animated', color="category",
             animation_group='category', 
             animation_frame=portfolio_category_composition_stats_combined['date'].astype(str), 
             range_x=[-10,25],
             orientation='h')
fig.show()